# Python Regex - greediness and overlapping matches

Learning regular expressions in Python in most cases starts with simple patterns for match, search or replace.
Once those basic patterns are second nature the more advanced patterns want to be mastered.
And for those a look into the regex matching algorithm is a good start.

The regex matching algorithm does
* greedy repetition qualifier matches
* non-overlapping matches

### imports and helpers

In [ ]:
import re

In [ ]:
class esc_high_colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class esc_std_colors:
    HEADER = '\033[35m'
    OKBLUE = '\033[34m'
    OKGREEN = '\033[32m'
    WARNING = '\033[33m'
    FAIL = '\033[31m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def p_groups(m):
    print m.group(), (m.start(), m.end())
    for ix, g in enumerate(m.groups()):
        print g, (m.start(ix+1), m.end(ix+1))
        
def p_matches(match_iter):
    for m in match_iter:
        p_groups(m)


def pf_group(group, start, end, esc_color=esc_std_colors.OKGREEN):
    print esc_color + group.rjust(end) + esc_std_colors.ENDC 
    
def pf_groups(m):
    print esc_std_colors.OKBLUE + m.string + esc_std_colors.ENDC

    pf_group(m.group(), m.start(), m.end(), esc_std_colors.WARNING)
    for ix, g in enumerate(m.groups()):
        pf_group(g, m.start(ix+1), m.end(ix+1))
        
def pf_matches(match_iter):
    for m in match_iter:
        pf_groups(m)
    print

## Greediness

Is the matching algorithm always greedy?

Greediness does not mean that for all patterns the algorithm proceeds to find another match and stops only at the end of the input string; as if the first match is never enough and it always looks for another one and another one.

For patterns with no repetition qualifiers (__*__, __+__, __?__, __{n,m}__) the match algorithm is not greedy. Once the first match was found the algorithm can stop and return the result.

### string search

A simple string search done with regex should not be greedy.
It should behave like the *find* method on the *String* object.

In [ ]:
"first blue second blue".find("blue")

In [ ]:
single_word = re.compile(r'blue')

In [ ]:
m = single_word.search("first blue second blue")
pf_groups(m)

### add repetition qualifiers to the regex

All repetition qualifiers are greedy.

Search for the first *a* or *b* in the input string and proceed until the first non-matching character. Be greedy.

In [ ]:
a_or_b = re.compile(r'[ab]+')

In [ ]:
pf_groups( a_or_b.search("caabaabcaaa") )

The greediness of the __.*__ regex can easily be underestimated.

In the below example the __.*__ pushes the __[ab]+__ pattern as far to the end of the string as possible.

In [ ]:
dot_star = re.compile(r'.*([ab]+)')

In [ ]:
pf_groups( dot_star.search("caabaabcaa") )

Another example.
Extract groups from a string. For example a name and number pair.

The regex looks for two groups:
* in the first group is the last letter of the first sequence of letters in the string which is followed, optionally separated by any character (no letters!), by at least one digit
* in the second group is the last digit of the first sequence of digits in the string which was preceeded, optionally separated by any character (no digits!), by at least one letter.

Important are the two __.\*__ in the regex.
They "steal" characters from the regex that follow them.
Leaving only the minimal number of characters, in this case one, to each of the two groups.

In [ ]:
letters_digits = re.compile(r'.*([a-z]+).*([0-9]+)')

In [ ]:
pf_groups( letters_digits.search("__abc__123__d") )

By removing the first (not required) greedy __.\*__ the first group now matches *abc*.

In [ ]:
letters_digits_less = re.compile(r'([a-z]+).*([0-9]+)')
pf_groups( letters_digits_less.search("__abc__123__d") )

Replacing the second greedy __.\*__ with "skip everything not matching the pattern of the first or second group".

In [ ]:
letters_digits_exclude = re.compile(r'([a-z]+)[^a-z0-9]*([0-9]+)')

In [ ]:
pf_groups( letters_digits_exclude.search("__abc__123__d") )

__Note__

Think in sequences and delimiters.
* Match sequences are the _match groups_, the substrings the regex should extract from the string.
* Delimiter sequences separate the match sequences from each other.

In the example *ZAZB* with
* Z = delimiter sequence
* A = name match sequence
* B = number match sequence

Make sure any greedy matching of delimiter sequences stops at the start of any match sequence.

extract all adjacent (name, number) pairs.

In [ ]:
letters_digits_exclude.findall("__abc__123__d_ef_45")

The next pattern is easier because it has defined delimiters **_+**.

The name and number pairs will be matched by the two groups:
* letters followed by at least one digit, optionally separated by delimiter
* digits preceeded by at least one letter, optionally separated by delimiter

The greediness of __[\_+]*__ removes delimiters of any length.

In [ ]:
letters_digits_delimiter = re.compile(r'[_+]*([a-z]+)[_+]*([0-9]+)')

In [ ]:
letters_digits_delimiter.findall("__abc__123__d")

In [ ]:
letters_digits_delimiter.findall("__abc__123__456__de__78")

In [ ]:
letters_digits_delimiter.findall("__abc123de45")

In [ ]:
pf_matches( letters_digits_delimiter.finditer("__abc123de45") )

## Overlapping Matches

The regex algorithm finds the first or all non-overlapping matches.

In [ ]:
only_a = re.compile(r'aa')

In [ ]:
only_a.findall("aaaa")

In [ ]:
pf_matches( only_a.finditer("aaaa") )

In [ ]:
m = only_a.search("aaa")
pf_groups(m)

### which strings qualify for overlapping matches?

#### match string of a repeated single letter

* "aaaaa" -> is a match
* "aabaa" -> not a match

In [ ]:
one = re.compile(r'^([a-z])(\1+)$')

In [ ]:
one.match("aaaaa").groups()

In [ ]:
one.findall("aabaa")

In [ ]:
len(set("aaaa")) == 1

#### match a string which repeats its start at the end

in general it is possible to find more than one solution.
for 2 solutions a match could be
* the shortest match or
* the longest match

In [ ]:
'''
aca
a a

abcab
ab ab

abcdabc
abc abc

ababcabab
ab     ab
abab abab

aacaa
a   a
aa aa

abcdeabcd
abcd abcd
'''

examples = {
    'basic': ["aa", "aca", "abcab", "abcdabc", "abcdeabcd",],
    'greedy_start': ["ababcabab", "aacaa",],
    'greedy_middle': ["abababccab",],
    'longest': ["abcdabcab", "abcdabcabab", "abcddddabcabab",],
}


def all_solutions(word):
    solutions = []
    for cut in xrange(len(word)/2 +1):
        if word[:cut] == word[-cut:]:
            solutions.append(word[:cut])
    return solutions

def all_solutions_comprehension(word):
    return [word[:cut] for cut in xrange(len(word)/2 +1) if word[:cut] == word[-cut:]]


for words in examples.values():
    for ex in words:
        print ex, all_solutions_comprehension(ex)

#### the shortest match

the greediness of the __+__ qualifier makes it impossible to find the shortest match with a single regex.

In [ ]:
start_repeats_at_end_shortest = re.compile(r'([a-z]+).*(\1)')

In [ ]:
pf_matches(start_repeats_at_end_shortest.finditer("ababcabab"))

#### the longest match

two greedy repetition qualifiers.
from the examples it seems
* same precedence
* left-associative

In [ ]:
start_repeats_at_end_longest = re.compile(r'([a-z]+).*(\1)')

easy to match strings.
a unique sequence at the start and it is only repeated once at the end.
optionally the unique start and end sequence is separated by one or more characters.

In [ ]:
for ex in examples['basic']:
    pf_matches(start_repeats_at_end_longest.finditer(ex))

it finds the first match for __[a-z]+__ and extends it greedily as long as it can find another identical match later in the string for the __(\\1)__ backreference.

In [ ]:
for ex in examples['greedy_start']:
    pf_matches(start_repeats_at_end_longest.finditer(ex))

the first and longest match for __([a-z]+)__ with a required backreference __(\\1)__ is __ab__.
it is not possible to extend the match of __([a-z]+)__ and still find a backreference.

the greediness of __.*__ matches until the last of all possible backrefences is reached.

In [ ]:
for ex in examples['greedy_middle']:
    pf_matches(start_repeats_at_end_longest.finditer(ex))

The *start_repeats_at_end_longest* has a bug.

it finds the first match for __[a-z]+__ and extends it greedily as long as it can find another identical match later in the string for the __(\\1)__ backreference.

the backreference can be anywhere later in the string. not only at the end of the string.
even more than one match within the string is possible.

In [ ]:
for ex in examples['longest']:
    pf_matches(start_repeats_at_end_longest.finditer(ex))

#### excursion - collect additional repeats of \1 following the match sequence at the front.

the following example has an additional __ab__ in between the identical start and end substrings.
without any modifications the *start_repeats_at_end_longest* regex does not match it.

In [ ]:
for m in start_repeats_at_end_longest.finditer("ababab"):
    pf_groups(m)

another group __(\1*)__ with a greedy repetition qualifier will collect any additional repeats after the start __A__.

In [ ]:
start_repeats_at_end = re.compile(r'([a-z]+)(\1*).*(\1+)')

In [ ]:
pf_matches(start_repeats_at_end.finditer("ababab"))

In [ ]:
p_matches(start_repeats_at_end.finditer("abababcab"))

In [ ]:
p_matches(start_repeats_at_end.finditer("ababcabab"))

In [ ]:
p_matches(start_repeats_at_end.finditer("abcababdabcab"))

### overlap is a constant substring of fixed length

there is one way to overlap.
clear distinction between the overlap pattern and the match pattern.
end sequence of one can be used as start sequence of an other match.
the pattern of what needs to be in front of the match is fixed.

the pattern is **ABA**.
overlap is possible on **A**.

This kind of overlap is best matched with _positive lookbehind assertion_.

In [ ]:
pattern = re.compile(r'(?<=--)(catchme)--')

In [ ]:
pattern.findall("--catchme--catchme--catchme--__catchme--")

### overlap is variable

the match pattern contains the overlap pattern.
the pattern of what needs to be in front of the match is variable.

Again the two cases of overlapping matches:

* repeated is one character or a sequence of characters, "aa", "abab", "abcabcabc".
* an identical start and end sequence is separated by a different sequence.

expressing the same content with the help of some symbols:

* the start sequence is __A__ and that sequence is the only sequence.
the patterns are __A__, __AA__, __AAA__, ..
there is more than one way to overlap for a given string.

* __A__ is again the start sequence and there is a separating sequence __B__.
the only pattern is __ABA__.
there is only one way to overlap for a given string.
to find the only solution several possible overlaps might need to be tried.

In [ ]:
'''
ababab
abab
  abab
A = ab

aaaaa
aa
 aa
  aa
   aa
A=a

ababababab
ababab
  ababab
    ababab
A=ab


aacaacaa
aacaa
   aacaa
A=aa
B=c

abcabcab
abcab
   abcab
A=ab
B=c

ababcababcabab
ababcabab
     ababcabab
A=abab
B=c

ababcabababcabab
ababcabab
       ababcabab
A=abab
B=c
'''

# (<pattern>, <data>)
overlap_examples = [("abab", "ababab"), ("aacaa", "aacaacaa"),
                    ("aa", "aaa"), ("abcab", "abcabcab"),
                    ("ababab", "ababababab"), ("aaaa", "aaaaaaa"),
                    ("abc", "aaabcaaaabc"),
                    ("abaaba", "abaababaaba")]


def run_examples(func):
    for ex in overlap_examples:
        pattern, data = ex
        print pattern, data
        pf_matches(func(pattern, data))

**n** is _len(data)_, **k** is _len(pattern)_, **m** is number of non-overlapping and overlapping matches in the string.

#### brute-force algorithm

iterate over the string left to right and check for a match beginning at the current position in the string.

roughly the cost is **n** function calls _re.match()_, for large **n** and small **k**.
the number of function calls do not depend on **m**, the number of matches in the string.
**n + m * k-1** reads.

In [ ]:
def findall_with_overlapping_forced(pattern, data):
    all_matches = []
    
    regex = re.compile(pattern)
    for ix in range(len(data)-len(pattern)+1):
        m = regex.match(data, ix)
        if m:
            all_matches.append(m)

    return all_matches

In [ ]:
run_examples(findall_with_overlapping_forced)

#### match-reset-match algorithm

iterate from search to search.
after every match reset the start of the search to the character after the *start position* of the match.

the cost is **m** function calls _re.search()_, **n + m * k-1** characters read.

In [ ]:
def findall_with_overlapping(pattern, data):
    all_matches = []
    
    regex = re.compile(pattern)
    m = regex.search(pattern)
    while m:
        all_matches.append(m)
        m = regex.search(data, m.start()+1)

    return all_matches

In [ ]:
run_examples(findall_with_overlapping)

#### match-overlap-match

generate all the possible overlap sequences in the match pattern.
find all the non-overlapping matches.
iterate over the non-overlapping matches and reset start position to *end of non-overlap match - length(overlap sequence)*

the cost is **m** function calls _re.search()_, **n + m * (length longest overlap)** characters read.

instead of going back to the first character after the start of an non-overlapping match, this algorithm moves back just the length of the longest possible overlap.

in case of a very long pattern and a short possible overlap this algorithm would read less characters.

__Note__

regex do not support recursive patterns.
recursion could find patterns where the pattern repeats itself within the pattern.

In [ ]:
def _overlap_for_aa(word):
    '''
    resursively search for single itself repeating substring
    abab -> [ab]
    aaa -> [a, aa]
    aba -> []
    '''
    sequence_aa = re.compile(r'^([a-z]+)(\1+)$')    
    
    def min_sequence(word):
        m = sequence_aa.search(word)
        if m:
            if len(m.group(1)) < len(m.group(2)):
                return min_sequence(m.group(1))
            else:
                return min_sequence(m.group(2))
        else:
            return word
    
    aa = min_sequence(word)
    repeats = (len(word) / len(aa) - 1)
    
    return [aa*i for i in range(repeats, 0, -1)] if aa != word else []


def _overlap_for_aba(word):
    '''
    recursively search for substring with identical start and end
    aba -> [a]
    abab -> [ab]
    aaa -> [a]
    ababcabab -> [ab, abab]
    acacaca -> [a, aca]
    '''
    sequence_aba = re.compile(r'^([a-z]+).*(\1)$')

    def recursive_aba(word):
        m = sequence_aba.search(word)
        if m:
            found = recursive_aba(m.group(1))
            found.append(m.group(1))
            return found
        else:
            return []
    
    return recursive_aba(word)


def find_overlap_sequences(word):
    '''
    create a list of all substrings that could be part of an overlapping match
    '''
    aa = _overlap_for_aa(word)
    aba = _overlap_for_aba(word)
    
    if aa and aba:
        if set(aba) <= set(aa):
            return aa
        else:
            return aba
    elif not aa and aba:
        return aba
    elif aa and not aba:
        raise AssertionError
    else:
        return []


def find_overlapping(data, pattern, overlaps, non_overlapping):
    '''
    try each possible overlap substring on every non-overlapping match
    '''
    overlapping = []
    regex = re.compile(pattern)
    
    for non_overlap in non_overlapping:
        for overlap in overlaps:    
            m = regex.match(data, non_overlap.end() - len(overlap))
            if m:
                overlapping.append(m)
    
    return overlapping


def findall_with_overlapping(pattern, data):
    
    overlap_seq = find_overlap_sequences(pattern)
    
    regex = re.compile(pattern)
    non_overlaps = [m for m in regex.finditer(data)]
    
    overlaps = []
    if overlap_seq and non_overlaps:
        overlaps = find_overlapping(data, pattern, overlap_seq, non_overlaps)
    
    return non_overlaps + overlaps

In [ ]:
run_examples(findall_with_overlapping)

## matching algorithm - open questions

expected matches __abab__ and __abab__

In [ ]:
repeat_ab = re.compile(r'(ab+).*(\1)')

In [ ]:
pf_matches( repeat_ab.finditer("ababcabab") )

changing repeated group expression to __(\\1+)__ gives same matches

In [ ]:
repeat_ab_plus = re.compile(r'(ab+).*(\1+)')

In [ ]:
pf_matches( repeat_ab_plus.finditer("ababcabab") )

__(ab)(\\1\*)__ is not equal to __(ab+)__

In [ ]:
repeat_ab_extra_group = re.compile(r'(ab)(\1*).*(\1)')

In [ ]:
pf_matches( repeat_ab_extra_group.finditer("ababcabab") )

__(ab)+__ is not equal to __(ab+)__

In [ ]:
repeat_ab_hard_in = re.compile(r'(ab+).*(ab+)')

In [ ]:
pf_matches( repeat_ab_hard_in.finditer("ababcabab") )

In [ ]:
repeat_ab_hard_out = re.compile(r'(ab)+.*(ab)+')

In [ ]:
pf_matches( repeat_ab_hard_out.finditer("ababcabab") )